In [1]:
import os, cv2, re, random
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array , load_img
from keras import layers, models, optimizers
from keras import backend as K
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
img_width=150
img_height=150
TRAIN_DIR='train/'
TEST_DIR='test1/'
train_images_dogs_cats = [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR)]
test_images_dogs_cats= [TEST_DIR+i for i in os.listdir(TEST_DIR)]

In [3]:
test_images_dogs_cats[0]

'test1/1.jpg'

In [4]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)' , text)]

In [5]:
train_images_dogs_cats.sort(key=natural_keys)
train_images_dogs_cats = train_images_dogs_cats [0:1300] + train_images_dogs_cats[12500:13800]
test_images_dogs_cats.sort(key=natural_keys)

In [11]:
train_images_dogs_cats[0]

'train/cat.0.jpg'

In [6]:
def prepare_data(list_of_images):
    
    x = [] # images as arrays
    y = [] # labels
    
    for image in list_of_images:
        x.append(cv2.resize(cv2.imread(image), (img_width,img_height), interpolation=cv2.INTER_CUBIC))
    
    for i in list_of_images:
        if 'dog' in i:
            y.append(1)
        elif 'cat' in i:
            y.append(0)
        #else:
            #print('neither cat nor dog name present in images')
            
    return x, y

In [7]:
X,Y = prepare_data(train_images_dogs_cats)
print(K.image_data_format())

channels_last


In [10]:
len(Y)

2600

In [19]:
# First split the data in two sets, 80% for training, 20% for Val/Test)
X_train, X_val, Y_train, Y_val = train_test_split(X,Y, test_size=0.2, random_state=1)

In [20]:
nb_train_samples = len(X_train)
nb_validation_samples = len(X_val)
batch_size = 16

In [22]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(32, (3, 3)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(64, (3, 3)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64))
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1))
model.add(layers.Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
activation_6 (Activation)    (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_7 (Activation)    (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 34, 34, 64)        18496     
__________

In [23]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [24]:
train_generator = train_datagen.flow(np.array(X_train), Y_train, batch_size=batch_size)
validation_generator = val_datagen.flow(np.array(X_val), Y_val, batch_size=batch_size)

In [25]:
history = model.fit_generator(
    train_generator, 
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size
)

Epoch 1/30
130/130 [==============================] - 55s 426ms/step - loss: 0.7172 - acc: 0.5226 - val_loss: 0.7366 - val_acc: 0.5020
Epoch 2/30
130/130 [==============================] - 53s 405ms/step - loss: 0.6730 - acc: 0.6236 - val_loss: 0.6386 - val_acc: 0.6627
Epoch 3/30
130/130 [==============================] - 53s 406ms/step - loss: 0.6381 - acc: 0.6572 - val_loss: 0.6104 - val_acc: 0.6587
Epoch 4/30
130/130 [==============================] - 53s 407ms/step - loss: 0.6141 - acc: 0.6827 - val_loss: 0.6324 - val_acc: 0.6627
Epoch 5/30
130/130 [==============================] - 52s 401ms/step - loss: 0.5961 - acc: 0.6986 - val_loss: 0.5505 - val_acc: 0.7242
Epoch 6/30
130/130 [==============================] - 53s 406ms/step - loss: 0.5712 - acc: 0.7159 - val_loss: 0.5433 - val_acc: 0.7421
Epoch 7/30
130/130 [==============================] - 53s 408ms/step - loss: 0.5517 - acc: 0.7231 - val_loss: 0.5533 - val_acc: 0.7262
Epoch 8/30
130/130 [==============================] - 5

In [26]:
model.save_weights('model_wieghts.h5')
model.save('model_keras.h5')

In [27]:
X_test, Y_test = prepare_data(test_images_dogs_cats) #Y_test in this case will be []

In [28]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [30]:
test_generator = val_datagen.flow(np.array(X_test), batch_size=batch_size)
prediction_probabilities = model.predict_generator(test_generator,steps=1, verbose=1)

1/1 [==============================] - 25s 25s/step


ValueError: `steps=None` is only valid for a generator based on the `keras.utils.Sequence` class. Please specify `steps` or use the `keras.utils.Sequence` class.